# Computing Fiber Diffraction Patterns

In [ ]:
from scripts import *

# An xyz file containing the atomic coordinates
# Hydrogen atoms do not need to be included in the xyz file
# In this example, we provide the coordinates of one triaminopyrimidine 
# and cyanuric acid with a 6-carbon chiral tail hexad unit

# Other available examples are 'geometries/TAP_CyCo6.xyz' and 'geometries/TAP_Cy.xyz'

coordinates_file = 'geometries/TAP_4MCyCo6.xyz'

# Specify the rise, twist, and number of hexad units in the stack
rise = 3.4 # Angstroms
twist = -26.67 # Degrees
number_of_hexads = 50
output_coordinate = 'fiber.xyz' # Save the generated geometry to this file

# Specify the rotation and tilt series
rotations = range(0, 360, 10) # Degrees. The rotation of the whole fiber around its axis
tilts = [6] # Degrees. The rotation of the whole fiber with respect to the incoming light beam

# Specify the fiber diffraction parameters
wavelength = 0.7749e-7 # The wavelength of the light beam in mm
distance_to_detector = 338.4 # The y-axis distance to the detector in mm
z_grid_limits = [-100.0, 100.0] # The limits of the grid along the z-axis in mm
x_grid_limits = [-100.0, 100.0] # The limits of the grid along the x-axis in mm
z_grid_size = 129 # The resolution of the grid along the z-axis
x_grid_size = 129 # The resolution of the grid along the x-axis
max_intensity_scaling = 0.01 # A scale factor to limit the maximum intensity in the diffraction plot

# Read the atomic coordinates
coords = np.loadtxt(coordinates_file, usecols=(1, 2, 3), skiprows=2)
atoms = np.loadtxt(coordinates_file, usecols=(0,), skiprows=2, dtype='str')

# Make a hexad helix; comment out if not using a hexad system
atoms, coords = helix_maker(atoms, coords, rise, twist, number_of_hexads)
save_xyz(atoms, coords, output_coordinate)

# Update coordinates and atomic numbers
coords *= 1e-7 # Convert from Angstroms to mm
atomic_numbers = [atomic_number[a] for a in atoms] # Convert from elements to atomic numbers

# Initialize diffraction data
diffraction_data = np.zeros((z_grid_size, x_grid_size))

# Obtain diffraction data for a rotational series around the fiber axis
for rotation in rotations:
    # Rotate around fiber axis
    coords = np.dot(Rz(rotation), coords.T).T
    
    # Obtain diffraction data for a tilt series
    for tilt in tilts:
        print('Rotation:', rotation, ", Tilt:", tilt)
        coords = np.dot(Ry(tilt), coords.T).T
        
        # Generate the fiber diffration
        # Here, we account for the rotation and tilt series by a simple sum of individual
        # diffraction patterns at each rotation-tilt value
        diffraction_data += generate_fiber_diffraction(atomic_numbers, coords, wavelength, distance_to_detector, 
                                                       z_grid_limits, x_grid_limits, z_grid_size, x_grid_size)

# Plot the fiber diffraction pattern
plot_fiber_diffraction(diffraction_data, z_grid_limits, x_grid_limits, max_intensity_scaling)